In [173]:
import pandas as pd
from sqlalchemy import create_engine 

In [174]:
# extract CSVs into DataFrames (NBA statistics)
nba_stats_path = "Resources/nba_extra.csv"
nba_stats_df = pd.read_csv(nba_stats_path)
nba_stats_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines/abrinal01,SG,24,OKC,75,8,1134,115,291,...,0.848,26,88,114,28,38,8,25,124,353
1,2,Quincy Acy/acyqu01,PF,27,BRK,70,8,1359,130,365,...,0.817,40,217,257,57,33,29,60,149,411
2,3,Steven Adams/adamsst01,C,24,OKC,76,76,2487,448,712,...,0.559,384,301,685,88,92,78,128,215,1056
3,4,Bam Adebayo/adebaba01,C,20,MIA,69,19,1368,174,340,...,0.721,118,263,381,101,32,41,66,138,477
4,5,Arron Afflalo/afflaar01,SG,32,ORL,53,3,682,65,162,...,0.846,4,62,66,30,4,9,21,56,179


In [175]:
# transformation 1 : drop null value columns to advoid errors
nba_stats_df.dropna(inplace = True)

In [176]:
# transformation 2 : split player name and remove
nba_stats_df[['Player_name','Player_name_2']]=nba_stats_df['Player'].str.split("/",expand=True)
nba_stats_df

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player_name,Player_name_2
0,1,Alex Abrines/abrinal01,SG,24,OKC,75,8,1134,115,291,...,88,114,28,38,8,25,124,353,Alex Abrines,abrinal01
1,2,Quincy Acy/acyqu01,PF,27,BRK,70,8,1359,130,365,...,217,257,57,33,29,60,149,411,Quincy Acy,acyqu01
2,3,Steven Adams/adamsst01,C,24,OKC,76,76,2487,448,712,...,301,685,88,92,78,128,215,1056,Steven Adams,adamsst01
3,4,Bam Adebayo/adebaba01,C,20,MIA,69,19,1368,174,340,...,263,381,101,32,41,66,138,477,Bam Adebayo,adebaba01
4,5,Arron Afflalo/afflaar01,SG,32,ORL,53,3,682,65,162,...,62,66,30,4,9,21,56,179,Arron Afflalo,afflaar01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,537,Tyler Zeller/zellety01,C,28,TOT,66,34,1109,187,334,...,195,305,47,15,35,47,126,441,Tyler Zeller,zellety01
659,537,Tyler Zeller/zellety01,C,28,BRK,42,33,703,125,229,...,131,194,28,8,21,35,78,300,Tyler Zeller,zellety01
660,537,Tyler Zeller/zellety01,C,28,MIL,24,1,406,62,105,...,64,111,19,7,14,12,48,141,Tyler Zeller,zellety01
661,538,Paul Zipser/zipsepa01,SF,23,CHI,54,12,824,81,234,...,118,131,46,20,15,43,86,218,Paul Zipser,zipsepa01


In [177]:
# transformation 3 : sort out necessary columns
nba_stats_selection = ["Player_name","Age","MP","FG%","3P%","2P%","eFG%","TRB","AST","STL","BLK","PTS"]
nba_stats_transformed_df = nba_stats_df[nba_stats_selection].copy()
nba_stats_transformed_df

,Player_name,Age,MP,FG%,3P%,2P%,eFG%,TRB,AST,STL,BLK,PTS
0,Alex Abrines,24,1134,0.395,0.380,0.443,0.540,114,28,38,8,353
1,Quincy Acy,27,1359,0.356,0.349,0.384,0.496,257,57,33,29,411
2,Steven Adams,24,2487,0.629,0.000,0.631,0.629,685,88,92,78,1056
3,Bam Adebayo,20,1368,0.512,0.000,0.523,0.512,381,101,32,41,477
4,Arron Afflalo,32,682,0.401,0.386,0.413,0.485,66,30,4,9,179
...,...,...,...,...,...,...,...,...,...,...,...,...
658,Tyler Zeller,28,1109,0.560,0.357,0.578,0.575,305,47,15,35,441
659,Tyler Zeller,28,703,0.546,0.385,0.567,0.568,194,28,8,21,300
660,Tyler Zeller,28,406,0.590,0.000,0.602,0.590,111,19,7,14,141
661,Paul Zipser,23,824,0.346,0.336,0.355,0.425,131,46,20,15,218


In [178]:
# transformation 4 : remove duplicate
nba_stats_groupby = nba_stats_transformed_df.groupby(['Player_name'])
nba_stats_final = nba_stats_groupby.mean()
nba_stats_final

,Age,MP,FG%,3P%,2P%,eFG%,TRB,AST,STL,BLK,PTS
Player_name,,,,,,,,,,,
Aaron Brooks,33.0,189.0,0.406,0.355,0.447,0.486,17.0,20.0,6.0,0.0,75.0
Aaron Gordon,22.0,1909.0,0.434,0.336,0.497,0.500,457.0,136.0,59.0,45.0,1022.0
Aaron Harrison,23.0,233.0,0.275,0.209,0.385,0.341,24.0,11.0,9.0,2.0,60.0
Aaron Jackson,31.0,35.0,0.333,0.250,0.400,0.389,3.0,1.0,0.0,0.0,8.0
Abdel Nader,24.0,522.0,0.336,0.354,0.321,0.413,71.0,26.0,15.0,10.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...
Zach Collins,20.0,1045.0,0.398,0.310,0.455,0.458,221.0,52.0,17.0,31.0,292.0
Zach LaVine,22.0,656.0,0.383,0.341,0.405,0.442,94.0,72.0,24.0,4.0,401.0
Zach Randolph,36.0,1508.0,0.473,0.347,0.503,0.507,397.0,127.0,42.0,10.0,857.0


In [179]:
# transformation 5 : rename the column headers

nba_stats_final = nba_stats_final.rename(columns={"MP": "minutes_played",
                                                  "FG%": "field_goal_success",
                                                  "3P%": "goal_success_3P",
                                                  "2P%": "goal_success_2P",
                                                  "eFG%":"effective_field_goal",
                                                  "TRB": "total_rebounds",
                                                  "AST": "assist",
                                                  "STL": "steal",
                                                  "BLK": "blocking",
                                                  "PTS": "points"
                                                 })
nba_stats_final

,Age,minutes_played,field_goal_success,goal_success_3P,goal_success_2P,effective_field_goal,total_rebounds,assist,steal,blocking,points
Player_name,,,,,,,,,,,
Aaron Brooks,33.0,189.0,0.406,0.355,0.447,0.486,17.0,20.0,6.0,0.0,75.0
Aaron Gordon,22.0,1909.0,0.434,0.336,0.497,0.500,457.0,136.0,59.0,45.0,1022.0
Aaron Harrison,23.0,233.0,0.275,0.209,0.385,0.341,24.0,11.0,9.0,2.0,60.0
Aaron Jackson,31.0,35.0,0.333,0.250,0.400,0.389,3.0,1.0,0.0,0.0,8.0
Abdel Nader,24.0,522.0,0.336,0.354,0.321,0.413,71.0,26.0,15.0,10.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...
Zach Collins,20.0,1045.0,0.398,0.310,0.455,0.458,221.0,52.0,17.0,31.0,292.0
Zach LaVine,22.0,656.0,0.383,0.341,0.405,0.442,94.0,72.0,24.0,4.0,401.0
Zach Randolph,36.0,1508.0,0.473,0.347,0.503,0.507,397.0,127.0,42.0,10.0,857.0


In [180]:
# extract CSVs into DataFrames (NBA salary)
nba_salary_path = "Resources/NBA_season1718_salary.csv"
nba_salary_df = pd.read_csv(nba_salary_path)
nba_salary_df

,Unnamed: 0,Player,Tm,season17_18
0,1,Stephen Curry,GSW,34682550.0
1,2,LeBron James,CLE,33285709.0
2,3,Paul Millsap,DEN,31269231.0
3,4,Gordon Hayward,BOS,29727900.0
4,5,Blake Griffin,DET,29512900.0
...,...,...,...,...
568,569,Quinn Cook,NOP,25000.0
569,570,Chris Johnson,HOU,25000.0
570,571,Beno Udrih,DET,25000.0
571,572,Joel Bolomboy,MIL,22248.0


In [181]:
# transformation 1 : drop null value columns to advoid errors
nba_salary_df.dropna(inplace = True)

In [182]:
# transformation 2 : sort out necessary columns
nba_salary_selection = ["Player","season17_18"]
nba_salary_transformed_df = nba_salary_df[nba_salary_selection].copy()
nba_salary_transformed_df

,Player,season17_18
0,Stephen Curry,34682550.0
1,LeBron James,33285709.0
2,Paul Millsap,31269231.0
3,Gordon Hayward,29727900.0
4,Blake Griffin,29512900.0
...,...,...
568,Quinn Cook,25000.0
569,Chris Johnson,25000.0
570,Beno Udrih,25000.0
571,Joel Bolomboy,22248.0


In [183]:
# transformation 3 : rename the column headers

nba_salary_final = nba_salary_transformed_df.rename(columns={"Player": "Player_name",
                                                  "season17_18": "Salary"
                                                 })
nba_salary_final

,Player_name,Salary
0,Stephen Curry,34682550.0
1,LeBron James,33285709.0
2,Paul Millsap,31269231.0
3,Gordon Hayward,29727900.0
4,Blake Griffin,29512900.0
...,...,...
568,Quinn Cook,25000.0
569,Chris Johnson,25000.0
570,Beno Udrih,25000.0
571,Joel Bolomboy,22248.0


In [187]:
connection_string = "postgres:postgres@127.0.0.1:3475/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [188]:
engine.table_names()

<ipython-input-188-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "127.0.0.1" and accepting
	TCP/IP connections on port 3475?

(Background on this error at: http://sqlalche.me/e/14/e3q8)